In [12]:
from PIL import Image
from colormath.color_objects import LabColor, sRGBColor
from colormath.color_conversions import convert_color
import numpy as np
import os
#import matplotlib.pyplot as plt

In [57]:
class pre_processing():
    def __init__(self,name,extension):
        self.name = name
        self.extension = extension
        self.image = Image.open(self.name+self.extension)
        self.processing()
    
    def processing(self):
        if(self.is_RGB()):
            self.cut()
            self.resize(128)
            self.save('resized')
            lab_ndarray = self.rgb_image2lab_ndarray()
            self.save_ndarray(lab_ndarray,'ndarray')
            
            
    def is_RGB(self):
        return (self.image.mode=='RGB')
    

    def cut(self):
        size = min(self.image.size)
        self.image = self.image.crop((0,0,size,size))
        self.size = size
        
    
    def resize(self,size):
        self.image = self.image.resize((size,size),Image.ANTIALIAS)
        self.size = 128

        
    def save(self,folder):
        directory = os.getcwd()+'/'+folder+'/'
        name = folder+'_'+self.name+self.extension
        self.image.save(directory+name)
        
    
    def colors_rgb2lab(self,colors):
        r, g, b = colors
        rgb = sRGBColor(r, g, b, is_upscaled=True)
        return convert_color(rgb,LabColor).get_value_tuple()
    
    
    def colors_lab2rgb(self,colors):
        l, a, b = colors
        lab = LabColor(l, a, b,illuminant='d65')
        return convert_color(lab, sRGBColor).get_upscaled_value_tuple()
    
    
    def slicing(self,array,n):
        return np.array(np.split(array,n))
    
    
    def save_ndarray(self,ndarray,folder):
        directory = os.getcwd()+'/'+folder+'/'
        name = folder+'_'+self.name+'.npy'
        np.save(directory+name,ndarray)
        
        
    def save_L_a_b_ndarrays(self,lab_list):
        transposed_lab_list = lab_list.T
        
        L_ndarray = self.slicing(transposed_lab_list[0],self.size)
        self.save_ndarray(L_ndarray,'L')

        a_ndarray = self.slicing(transposed_lab_list[1],self.size)        
        b_ndarray = self.slicing(transposed_lab_list[2],self.size)
        ab_ndarray = np.array([a_ndarray,b_ndarray])
        self.save_ndarray(ab_ndarray,'ab')
    
    
    def rgb_image2lab_ndarray(self):
        rgb_list = np.array(self.image.getdata())
        lab_list = np.apply_along_axis(self.colors_rgb2lab, 1, rgb_list)
        lab_ndarray = self.slicing(lab_list,self.size)
        
        self.save_L_a_b_ndarrays(lab_list)

        return lab_ndarray
    
    
    def lab_ndarray2rgb_image(self,ndarray):
        lab_list = ndarray.reshape((self.size**2),3)
        rgb_list = np.apply_along_axis(self.colors_lab2rgb, 1, lab_list)
        rgb_array = self.slicing(rgb_list,self.size)
        
        return Image.fromarray(rgb_array.astype('uint8'),'RGB')
    
    
    def load_ndarray(self,name):
        return np.load(name)

    

In [58]:
if __name__ == "__main__":
    arq = open('names_list.txt', 'r')
    texto = arq.read()
    arq.close()
    names_list = texto.split('.jpeg\n')
    del(names_list[len(names_list)-1])
    
    print(len(names_list))
    
    print(names_list[0])
    
    for name in names_list:
       im = pre_processing(name,'.jpeg')
    
    print('finish')

FileNotFoundError: [Errno 2] No such file or directory: 'names_list.txt'